In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import transformers
import os

In [2]:
# MODEL

_LANGUAGE_         = 'es'
_PRETRAINED_LM_    = 'pysentimiento/robertuito-base-cased'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig(mh_adapter = True, reduction_factor = 32)
_MAX_SEQ_LEN_      = 128
_OUTPUT_DIR_       = 'adapter_checkPoints'
_LOGGING_STEPS_    = 2
_NUM_AUTHORS_      = [8, 16, 32, 64, 128, 256, 512]
_K_FOLD_CV_        = 5

# TRAIN

_NO_GPUS_          = 2
_BATCH_SIZE_       = int(32 / _NO_GPUS_)
_EPOCHS_           = {'gender': 20, 'variety': 20}
_LEARNING_RATE_    = 5e-5

## Base

In [3]:
# LABEL DICTONARIES -----------------------------------------------------------------------

gender_dict    = {'female': 0, 
                  'male':   1}

varietyEN_dict = {'australia'    : 0,
                  'canada'       : 1,
                  'great britain' : 2,
                  'ireland'      : 3,
                  'new zealand'   : 4,
                  'united states': 5}

varietyES_dict = {'argentina': 0,
                  'chile'    : 1,
                  'colombia' : 2,
                  'mexico'   : 3,
                  'peru'     : 4,
                  'spain'    : 5,
                  'venezuela': 6}    

In [4]:
# SET LANGUAGE DIRECTORY

if _LANGUAGE_ == 'en':
    variety_dict = varietyEN_dict

elif _LANGUAGE_ == 'es':
    variety_dict = varietyES_dict

In [5]:
# SET LANGUAGE TOKENIZER
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

## Dataset

In [6]:
from DatasetPAN17 import BasePAN17, DatasetPAN17, DatasetCrossVal

In [7]:
baseTrain  = BasePAN17(Dir             = '../data',
                      split            = 'train',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)

baseTest  = BasePAN17(Dir             = '../data',
                      split            = 'test',
                      language         = _LANGUAGE_,
                      tokenizer        = tokenizer,
                      gender_dict      = gender_dict,
                      variety_dict     = variety_dict,
                      tweet_batch_size = _TWEET_BATCH_SIZE_,
                      max_seq_len      = _MAX_SEQ_LEN_,
                      preprocess_text  = _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 84000


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 56000



In [8]:
Test = DatasetPAN17(Base_Dataset = baseTest, label = 'gender')

## Training

In [9]:
tasks = ['gender']

In [10]:
from transformers import AutoModelForSequenceClassification, AutoAdapterModel
from transformers import TrainingArguments, Trainer, AdapterTrainer, EarlyStoppingCallback
from TestingPAN17 import compute_accuracy, compute_test_metrics

num_v           = len(baseTest.variety_dict)
num_labels_dict = {"gender": 2, "variety": num_v,}

FewShot_Results = {}

for num in _NUM_AUTHORS_:
    # SHOW CURRENT PORTION
    print("Working with " + str(num) + " authors per label ... ")
    
    dataset_dict = {}
    models = {}
    
    for task_name in tasks:
        
        # INITIALIZE MODEL-----------------------------------------
        
        models[task_name] = AutoAdapterModel.from_pretrained(_PRETRAINED_LM_)
        
        
        acc = []
        f1s = []
        
        for val_idx in range(_K_FOLD_CV_):
            print("Train,Val split number " + str(val_idx + 1) + " of " + str(_K_FOLD_CV_))
        
            # INITIALIZE ADAPTER-----------------------------------------
            
            models[task_name].add_adapter(
                adapter_name = task_name,
                config       = _ADAPTER_CONFIG_
            )
            
            models[task_name].add_classification_head(
                head_name    = task_name,
                num_labels   = num_labels_dict[task_name],
            )
            
            models[task_name].set_active_adapters(task_name)
            models[task_name].train_adapter(task_name)
            
            # GENERATES DATASET WITH CURRENT PORTION ----------------------
            
            data_train, data_val = baseTrain.cross_val(k = _K_FOLD_CV_, val_idx = val_idx, num_authors = num)
            Train = DatasetCrossVal(Base_Data = data_train, label = 'gender')
            Val   = DatasetCrossVal(Base_Data = data_val  , label = 'gender')

            # TRAIN ADAPTER--------------------------------------------

            training_args = TrainingArguments(
                learning_rate               = _LEARNING_RATE_,
                num_train_epochs            = _EPOCHS_[task_name],
                per_device_train_batch_size = _BATCH_SIZE_,
                per_device_eval_batch_size  = _BATCH_SIZE_,
                output_dir                  = _OUTPUT_DIR_ + '/' + task_name,
                save_total_limit            = 5,
                overwrite_output_dir        = True,
                remove_unused_columns       = False,
                evaluation_strategy         = 'epoch',
                logging_strategy            = 'epoch',
                save_strategy               = 'epoch',
                metric_for_best_model       = 'eval_acc',
                load_best_model_at_end      = True,
            )

            trainer = AdapterTrainer(
                model           = models[task_name],
                args            = training_args,
                train_dataset   = Train,
                eval_dataset    = Val,
                compute_metrics = compute_accuracy,
                callbacks       = [EarlyStoppingCallback(early_stopping_patience = 2, early_stopping_threshold = 0.0001)]
            )
            trainer.args._n_gpu = _NO_GPUS_

            trainer.train()

            # TEST MODEL ------------------------------------

            results = trainer.predict(Test)
            metrics = compute_test_metrics(baseTest, results.predictions, 'gender')
            
            acc.append(metrics['accuracy'])
            f1s.append(metrics['f1-score'])
            
            # DELETE ADAPTER --------------------------------
            
            models[task_name].delete_adapter(task_name)
            models[task_name].delete_head(task_name)
        
        acc = np.array(acc)
        f1s = np.array(f1s)
        
        FewShot_Results[num] = {'accuracy': [acc.mean(), acc.std()], 'f1-score': [f1s.mean(), f1s.std()]}
        print("Results with " + str(num) + " authors per label: ", FewShot_Results[num])

Working with 8 authors per label ... 


Some weights of the model checkpoint at pysentimiento/robertuito-base-cased were not used when initializing RobertaAdapterModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at pysentimiento/robertuito-base-cased and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able

Train,Val split number 1 of 5


/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 320
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 200
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Acc
1,0.640600,0.665384,0.596071
2,0.521300,0.663332,0.603274
3,0.415700,0.689176,0.603333
4,0.336000,0.711153,0.610952
5,0.266200,0.754629,0.611726
6,0.243500,0.817822,0.613512
7,0.191700,0.832711,0.613810
8,0.165600,0.882409,0.614940
9,0.133800,0.955413,0.611667
10,0.107900,0.991520,0.613988


...

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20480
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12800
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functi

Epoch,Training Loss,Validation Loss,Acc
1,0.575700,0.607603,0.690833
2,0.521400,0.577315,0.697500
3,0.486800,0.619093,0.691369
4,0.456800,0.641859,0.686429


***** Running Evaluation *****
  Num examples = 16800
  Batch size = 32
Saving model checkpoint to adapter_checkPoints/gender/checkpoint-640
Configuration saved in adapter_checkPoints/gender/checkpoint-640/gender/adapter_config.json
Module weights saved in adapter_checkPoints/gender/checkpoint-640/gender/pytorch_adapter.bin
Configuration saved in adapter_checkPoints/gender/checkpoint-640/gender/head_config.json
Module weights saved in adapter_checkPoints/gender/checkpoint-640/gender/pytorch_model_head.bin
Configuration saved in adapter_checkPoints/gender/checkpoint-640/gender/head_config.json
Module weights saved in adapter_checkPoints/gender/checkpoint-640/gender/pytorch_model_head.bin
/001/usuarios/isaac.bribiesca/anaconda3/envs/NLP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, bu

acc: 0.7885714285714286: 100%|█████████████████████████████████████████████████████| 2800/2800 [00:17<00:00, 158.85it/s]


Results with 512 authors per label:  {'accuracy': [0.7906428571428572, 0.010267841614895116], 'f1-score': [0.7873580677573417, 0.016908853577989186]}
